In [15]:
def remove_header_footer(text):
    """
    Removes headers and footers from Project Gutenberg texts.
    """
    cleaned_text = []
    flag = False
    start = "*** START OF"
    end = "*** END OF"
    for line in text:
        if (start in line) and (flag == False):
            flag = True
        elif (end in line) and (flag == True):
            flag = False
        elif flag == True:
            for word in line.split():
                cleaned_text.append(word)
        else:
            pass
    return cleaned_text

In [7]:
import os

path = "C:\\Users\\mjcor\\Desktop\\ProgrammingStuff\\ThinkPython"
os.chdir(path)

In [16]:
head = open('heading_test.txt', "r")
remove_header_footer(head)

['this', 'is', 'only', 'a', 'test.']